In [5]:
# autolabel steel


import numpy as np
import sys
sys.path.insert(4, r'C:/Users/Asc-user/Documents/YOLO/darknet')
from darknet_noWinmode import *
import cv2
import glob
import time 
import copy
import os
import signal

def darknet_helper(image, width, height):
    darknet_image = make_image(width, height, 3)
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (width, height),interpolation=cv2.INTER_LINEAR)
    img_height, img_width, _ = image.shape
    width_ratio = img_width/width
    height_ratio = img_height/height
    copy_image_from_bytes(darknet_image, img_resized.tobytes())
    detections = detect_image(network, class_names, darknet_image, thresh=0.5)
    free_image(darknet_image)

    return detections, width_ratio, height_ratio
       

network, class_names, class_colors = load_network(r"C:/Users/Asc-user/Documents/YOLO/Y562_train/steel_train/yolov4-tiny.cfg", 
                                                  r"C:/Users/Asc-user/Documents/YOLO/Y562_train/steel_train/obj.data", 
                                                  r"C:/Users/Asc-user/Documents/YOLO/Y562_2PLCM/shid_paint_v1.weights")


def rotate_bbox(bbox, angle, image_width, image_height):
    # 將bounding box旋轉回原圖座標
    x, y, w, h = bbox
    cx, cy = x + w / 2, y + h / 2
    new_cx = cx * np.cos(angle) - cy * np.sin(angle)
    new_cy = cx * np.sin(angle) + cy * np.cos(angle)
    new_w = w * abs(np.cos(angle)) + h * abs(np.sin(angle))
    new_h = w * abs(np.sin(angle)) + h * abs(np.cos(angle))
    
    # 根據旋轉角度調整座標，考慮圖片的長寬不是正方形的情況
    new_x = new_cx - new_w / 2
    new_y = new_cy - new_h / 2
    new_x = max(0, min(new_x, image_width - new_w))
    new_y = max(0, min(new_y, image_height - new_h))
    new_w = max(0, min(new_w, image_width - new_x))
    new_h = max(0, min(new_h, image_height - new_y))
    
    return [new_x, new_y, new_w, new_h]


def detect_and_write(image, width, height, path):
    size = [image.shape[1], image.shape[0]]
    detections, width_ratio, height_ratio = darknet_helper(image, width, height)
    detections = sorted(detections, reverse=False, key=lambda s: s[2][1])
    print(detections)
    
    # 增加兩個flag來檢查是否成功辨識'steel'和'sid'
    has_steel = False
    has_sid = False
    
    for label, confidence, bbox in detections:
        left, top, right, bottom = bbox2points(bbox)
        left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
        top, bottom, left, right = abs(top), abs(bottom), abs(left), abs(right)
        if label == 'steel':
            label = '2'
        elif label == 'sid':
            label = '0'
        else:
            label = '1'
            
        dw = 1. / size[0]
        dh = 1. / size[1]
        x = (left + right) / 2.0
        y = (top + bottom) / 2.0
        w = right - left
        h = bottom - top
        x = x * dw
        w = w * dw
        y = y * dh
        h = h * dh
        
        write = open(path +'.txt', 'a')
        print(label, x, y, w, h, file=write)
        write.close() 


path = 'C:/Users/Asc-user/Documents/YOLO/Y562_train/shid_paint_2/'
for level_1 in os.listdir(path):
    image = cv2.imread(path + level_1)
    original_path = path + level_1[:-4]
    rotated_path_90 = path + level_1[:-4]
    rotated_path_270 = path + level_1[:-4]
    
    # 進行第一次辨識
    if detect_and_write(image, 416, 416, original_path):
        continue
    
    # 若'sid'沒有被辨識出來，則進行90度旋轉再辨識
    image_rotated_90 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    if detect_and_write(image_rotated_90, 416, 416, rotated_path_90):
        continue
    
    # 若'sid'還是沒有被辨識出來，則進行270度旋轉再辨識
    image_rotated_270 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    detect_and_write(image_rotated_270, 416, 416, rotated_path_270)
    
print('done')







[('steel', '93.01', (212.105712890625, 239.40118408203125, 233.81568908691406, 356.27423095703125))]
[]
[]
[('steel', '93.01', (212.105712890625, 239.40118408203125, 233.81568908691406, 356.27423095703125))]
[]
[]
[('steel', '93.01', (212.105712890625, 239.40118408203125, 233.81568908691406, 356.27423095703125))]
[]
[]
[('steel', '93.01', (212.105712890625, 239.40118408203125, 233.81568908691406, 356.27423095703125))]
[]
[]
[('hid', '57.11', (238.44842529296875, 141.88272094726562, 92.76374053955078, 78.02716064453125)), ('steel', '95.66', (214.7876739501953, 239.3670654296875, 218.93824768066406, 335.16839599609375))]
[]
[('sid', '66.94', (379.26776123046875, 55.17251968383789, 29.638904571533203, 79.45402526855469)), ('steel', '52.17', (208.9507598876953, 207.0657958984375, 237.3134307861328, 326.88275146484375))]
[('hid', '57.11', (238.44842529296875, 141.88272094726562, 92.76374053955078, 78.02716064453125)), ('steel', '95.66', (214.7876739501953, 239.3670654296875, 218.93824768066

AttributeError: 'NoneType' object has no attribute 'shape'